In [38]:
!pip install ipynb

In [39]:
import numpy as np
import os
from time import time
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import seaborn as sns
import ipynb
sns.set(style="darkgrid")
import warnings
#warnings.simplefilter(action='ignore', category=IntegrationWarning)

In [40]:
from ipynb.fs.full.closed_form_solution import f, p, p1, p2, call_price

In [41]:
start = time()

# Set the parameters
sigma = 0.61
theta = 0.019
kappa = 6.21
mu = 0.5
rho = -0.5
generator = np.random.default_rng()

n = [250, 400] # n time periods
m = [x**2 for x in n] # number of simulations

In [42]:
K = [1.5, 2.5]
S0 = [2, 3]
V0 = [0.01, 0.07]
int_rates = [0, 0.05]
time_maturity = [1, 2, 5]

Stock prices simulation:

In [43]:
def stock_price_generator (T, n ,m, r, S0, k, V0, sigma, theta, kappa, rho, generator):
    dt = T / n
    
    # Brownian motions:
    dw_v = generator.normal(size=(m, n)) * np.sqrt(dt)
    dw_i = generator.normal(size=(m, n)) * np.sqrt(dt)
    #dw_v = np.random.normal(size=(m, n)) * np.sqrt(dt)
    #dw_i = np.random.normal(size=(m, n)) * np.sqrt(dt)

    dw_s = rho * dw_v + np.sqrt(1.0 - rho ** 2) * dw_i

    # Perform time evolution 
    s = np.empty((m, n + 1)) # initialisation stock prices vector
    s[:, 0] = S0

    v = np.ones(m) * V0

    for t in range(n):
        dv = kappa * (theta - v) * dt + sigma * np.sqrt(v) * dw_v[:, t]
        ds = r * s[:, t] * dt + np.sqrt(v) * s[:, t] * dw_s[:, t]

        v = np.clip(v + dv, a_min=0.0, a_max=None)
        s[:, t + 1] = s[:, t] + ds
        
        
    return s[:,-1]
    

In [44]:
#s = stock_price_generator (, n ,m, r, S0, k, V0, sigma, theta, kappa, rho)

Plots stock prices simulations:

In [45]:
#plot_mc (s,,n)

Payoff:

In [46]:
# function which finds the expected payoff 
def find_expected_payoff(stock_paths, k, r, T):
    final = stock_paths - k
    payoff = [max(x,0) for x in final] # one payoff for each simulation
    payoff = np.asarray(payoff)
    expected_payoff = payoff.mean()
    c = expected_payoff * np.exp(-r * T)     # in case r=0, this step is useless
    
    return c

In [47]:
#c = find_expected_payoff(s, k, r, ) # in case r=0, this step is useless
#c

In [48]:
df = pd.DataFrame(columns=['S0', 'K', 'V0', 'T', 'r', 'n', 'm', 'closed_solution', 'mc_price', 'ST_std'])

In [49]:
# fill the dataset
for s0 in S0:
    print(f's0 {s0}')
    for k in K:
        print(f'k {k}')
        for v0 in V0:
            print(f'v0 {v0}')
            for t in time_maturity:
                print(f't {t}')

                for r in int_rates:
                    # analytical closed sol
                    sol = call_price(kappa, theta, sigma, rho, v0, r, t, s0, k)

                    for i in range(len(n)):
                        nn = n[i]
                        mm = m[i]

                        # monte carlo solution:
                        ST = stock_price_generator(
                            t, nn, mm, r, s0, k, v0, sigma, theta, kappa, rho,
                            generator)  # stock prices of mc simulation

                        mc_price = find_expected_payoff(ST, k, r, t)

                        new_row = {
                            'S0': s0,
                            'K': k,
                            'V0': v0,
                            'T': t,
                            'V0': v0,
                            'r': r,
                            'n': nn,
                            'm': mm,
                            'closed_solution': sol,
                            'mc_price': mc_price,
                            'ST_std': np.std(ST)
                        }
                        
                        # append row to the dataframe
                        df = df.append(new_row, ignore_index=True)


s0 2
k 1.5
v0 0.01
t 1
t 2
t 5


ipynb.fs.full.closed_form_solution:46: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.


v0 0.07
t 1
t 2
t 5
k 2.5
v0 0.01
t 1
t 2
t 5
v0 0.07
t 1
t 2
t 5
s0 3
k 1.5
v0 0.01
t 1
t 2
t 5
v0 0.07
t 1
t 2
t 5
k 2.5
v0 0.01
t 1
t 2
t 5
v0 0.07
t 1
t 2
t 5


In [51]:
df.head()

,S0,K,V0,T,r,n,m,closed_solution,mc_price,ST_std
0,2.0,1.5,0.01,1.0,0.00,250.0,62500.0,0.501704,0.505418,0.260141
1,2.0,1.5,0.01,1.0,0.00,400.0,160000.0,0.501704,0.505640,0.259024
2,2.0,1.5,0.01,1.0,0.05,250.0,62500.0,0.605693,0.575393,0.274204
3,2.0,1.5,0.01,1.0,0.05,400.0,160000.0,0.605693,0.577484,0.272457
4,2.0,1.5,0.01,2.0,0.00,250.0,62500.0,0.644682,0.517993,0.379576


In [52]:
df.to_csv("solution_complete_mc.csv")

In [53]:
def plot_mc (s,T,n, m):
    # Setup figure
    plt.figure(figsize=(8,6))
    # noinspection PyTypeChecker
    ax_lines = plt.axes()

    maxx = np.max(s)
    minn = np.min(s)

    # Make the line plots
    t = np.linspace(0, T, num = n + 1)
    ns = 40
    for i in range(ns):
        ax_lines.plot(t, s[i, :], lw=1.0)
    ax_lines.set(xlabel='Years', ylabel='St',title='Price Simulations')
    ax_lines.set_xlim((0, T))
    ax_lines.set_ylim((minn, maxx))

    # Add mean value to line plots
    ax_lines.plot([0.0, 1.0], [s[:, -1].mean(), s[:, -1].mean()], lw='2', ls="--", label='mean')
    plt.show()

    # plot stock prices distributions
    plt.figure(figsize=(6,5))
    bins = np.arange(1.4, 3, .04)
    plt.hist(s[:, -1], bins=bins)
    plt.xlabel('ST')
    plt.ylabel('Number')
    plt.title('Stock Prices distribution')
    plt.show()

In [58]:
solution_mc = pd.read_csv('solution_complete_mc.csv')
solution_mc.head()

,Unnamed: 0,S0,K,V0,T,r,n,m,closed_solution,mc_price,ST_std
0,0,2.0,1.5,0.01,1.0,0.00,250.0,62500.0,0.501704,0.505418,0.260141
1,1,2.0,1.5,0.01,1.0,0.00,400.0,160000.0,0.501704,0.505640,0.259024
2,2,2.0,1.5,0.01,1.0,0.05,250.0,62500.0,0.605693,0.575393,0.274204
3,3,2.0,1.5,0.01,1.0,0.05,400.0,160000.0,0.605693,0.577484,0.272457
4,4,2.0,1.5,0.01,2.0,0.00,250.0,62500.0,0.644682,0.517993,0.379576


In [59]:
len(solution_mc)

96

# AV

In [72]:
def stock_price_generator_av (T, n ,m, r, S0, k, V0, sigma, theta, kappa, rho, separate=False):
    dt = T / n
    
    m = int(m/2) # with av we can do half of the simulations
    n = int(n)
    
    # Brownian motions:
    dw_v = generator.normal(size=(m, n)) * np.sqrt(dt)
    dw_i = generator.normal(size=(m, n)) * np.sqrt(dt)

    dw_s = rho * dw_v + np.sqrt(1.0 - rho ** 2) * dw_i

    # Perform time evolution 
    s = np.empty((m, n + 1)) # initialisation stock prices vector
    s[:, 0] = S0
    
    s_ant = np.empty((m, n + 1))
    s_ant[:, 0] = S0

    v = np.ones(m) * V0
    v_ant = np.ones(m) * V0

    for t in range(n):
        dv = kappa * (theta - v) * dt + sigma * np.sqrt(v) * dw_v[:, t]
        dv_ant = kappa * (theta - v_ant) * dt + sigma * np.sqrt(v_ant) * dw_v[:, t]
        ds = r * s[:, t] * dt + np.sqrt(v) * s[:, t] * dw_s[:, t]
        ds_ant = r * s_ant[:, t] * dt + np.sqrt(v_ant) * s_ant[:, t] * dw_s[:, t]

        v = np.clip(v + dv, a_min=0.0, a_max=None)
        v_ant = np.clip(v_ant + dv_ant, a_min=0.0, a_max=None)
        
        s[:, t + 1] = s[:, t] + ds
        s_ant[:, t + 1] = s_ant[:, t] + ds_ant
        
    if separate==True:
        return s, s_ant
        
    return np.concatenate((s, s_ant))[:,-1]

In [73]:
mc_price_av = []
ST_av_std = []
i=0

for row in solution_mc.index:
    i = i+1
    if i % 10 == 0:
      print(i)

    T = solution_mc['T'][row]
    n = solution_mc['n'][row]
    m = solution_mc['m'][row]
    r = solution_mc['r'][row]
    S0 = solution_mc['S0'][row]
    K = solution_mc['K'][row]
    V0 = solution_mc['V0'][row]
    
    ST_ant = stock_price_generator_av (T, n, m, r, S0, K, V0, sigma, theta, kappa, rho)
    
    ST_av_std.append(np.std(ST_ant))
    mc_price_av.append(find_expected_payoff(ST_ant, K, r, T))


10
20
30
40
50
60
70
80
90


In [74]:
solution_mc['mc_price_av'] = mc_price_av
solution_mc['ST_av_std'] = ST_av_std

In [75]:
solution_mc.head()

,Unnamed: 0,S0,K,V0,T,r,n,m,closed_solution,mc_price,ST_std,mc_price_av,ST_av_std
0,0,2.0,1.5,0.01,1.0,0.00,250.0,62500.0,0.501704,0.505418,0.260141,0.505318,0.261869
1,1,2.0,1.5,0.01,1.0,0.00,400.0,160000.0,0.501704,0.505640,0.259024,0.503791,0.260026
2,2,2.0,1.5,0.01,1.0,0.05,250.0,62500.0,0.605693,0.575393,0.274204,0.578765,0.275233
3,3,2.0,1.5,0.01,1.0,0.05,400.0,160000.0,0.605693,0.577484,0.272457,0.576445,0.272337
4,4,2.0,1.5,0.01,2.0,0.00,250.0,62500.0,0.644682,0.517993,0.379576,0.518675,0.379652


In [80]:
standard_deviation = solution_mc[['ST_std', 'ST_av_std']]
standard_deviation.head()

,ST_std,ST_av_std
0,0.260141,0.261869
1,0.259024,0.260026
2,0.274204,0.275233
3,0.272457,0.272337
4,0.379576,0.379652


In [76]:
prices = solution_mc[['closed_solution', 'mc_price', 'mc_price_av']]
prices.head()

,closed_solution,mc_price,mc_price_av
0,0.501704,0.505418,0.505318
1,0.501704,0.505640,0.503791
2,0.605693,0.575393,0.578765
3,0.605693,0.577484,0.576445
4,0.644682,0.517993,0.518675


In [77]:
rmse_mc = np.sqrt(((prices.closed_solution - prices.mc_price)**2).sum())
rmse_mc_av = np.sqrt(((prices.closed_solution - prices.mc_price_av)**2).sum())

In [78]:
rmse_mc

1.272573108647589

In [79]:
rmse_mc_av

1.2764783391883372

In [50]:
solution_mc.to_csv("solution_complete_mc.csv")